# <center> CREDIT SCORING

<CENTER> -by Arya Octavian

------

# IMPORTING DATA

## Libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 99)
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from matplotlib import rcParams

matplotlib.rcParams['font.family'] = 'Tw Cen MT'
rcParams['figure.figsize'] = (12, 8)
rcParams['lines.linewidth'] = 3
rcParams['axes.titlesize'] = 'x-large'
rcParams['xtick.labelsize'] = 'medium'
rcParams['ytick.labelsize'] = 'medium'
rcParams['axes.facecolor'] = '1'

## Datasets

In [ ]:
df_raw_train = pd.read_csv("train.csv")
df_raw_train

In [ ]:
df_raw_test = pd.read_csv("test.csv")
df_raw_test

### Datasets Properties

In [ ]:
print('raw train:', df_raw_train.shape)
print('raw test:', df_raw_test.shape)

In [ ]:
print('-' * 50)
display(df_raw_train.info())
print('=' * 50)
display(df_raw_test.info())
print('-' * 50)

- It can be confirmed that the `Credit_Score` feature is the target for the machine learning to be created.

In [ ]:
print('-' * 50)
print("Train Data Describe:")
display(df_raw_train.describe())
print('=' * 50)
print("Test Data Describe:")
display(df_raw_test.describe())
print('-' * 50)

Note:
- There are unusual values in the `Num_Bank_Accounts` feature, namely (-). This is suspected to be due to human error or representation of Null values.
- The data will be cleaned in the upcoming steps.

In [ ]:
print('-' * 50)
print("Train Data Categorical Columns Describe:")
display(df_raw_train.describe(include=['object']).T)
print('=' * 50)
print("Test Data Categorical Columns Describe:")
display(df_raw_test.describe(include=['object']).T)
print('-' * 50)

Note:
- There are many unusual values in some categorical columns, such as excessive use of symbols. Of course, those values will be deleted or replaced with other values.
- The data will be cleaned in the upcoming steps.

In [ ]:
num_different_cols = sum(df_raw_train.columns[:-1] != df_raw_test.columns)
num_different_cols

- The output above indicates the number, names, and data types of columns that have no differences.

# DATA CLEANING

To expedite the data cleaning process, the dataset and training data will be merged first. The data can be separated later considering that the test data in the evaluation process will still be taken from the train dataset, NOT from the test dataset which has null `Credit_Score` target features.

In [ ]:
df = pd.concat([df_raw_train, df_raw_test], ignore_index=True)
df.shape

## Check Missing Values

In [ ]:
df.isna().sum().sort_values(ascending=False)

## Check Invalid Values in Object Datatype Columns

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
object_columns

In [ ]:
print(f"Unique values by features\n")

for object_unique in object_columns:
    unique_values = df[object_unique].unique()
    print(f"{object_unique}:\n {unique_values}\n")

### Handling Invalid Values

In [ ]:
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')).replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)

- Check if the function above works properly.

In [ ]:
print('-' * 50)
display(df_raw_test.loc[[49997]])
display(df_raw_train.loc[[99999]])
print('=' * 50)
display(df.loc[[149997, 99999]])
print('-' * 50)

In [ ]:
df.isna().sum().sort_values(ascending=False)

- Before filling null values, it is very important to change the data type according to the real-world context.

## Changing Data Type

In [ ]:
df.select_dtypes(include=['object']).info()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['ID'].unique()

- Values in the `ID` feature are strings in hexadecimal format, so they need to be converted to integers.

In [ ]:
df['ID'] = df['ID'].apply(lambda x : int(x, 16))
df['ID'].unique()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['Customer_ID'].unique()

- The value in the 'Customer_ID' feature is also in hexadecimal format, but with the prefix 'CUS_' added to it. Therefore, the prefix needs to be removed first using indexing.

In [ ]:
df['Customer_ID'] = df['Customer_ID'].apply(lambda x : int(x[4:], 16))
df['Customer_ID'].unique()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['Month'].unique()

- Convert categorical values to numbers representing the order of the month in a year.

In [ ]:
df['Month'] = pd.to_datetime(df['Month'], format='%B').dt.month
df['Month'].unique()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['Age'].unique()

- The age values should be in integer format instead of string format.

In [ ]:
df['Age'] = df.Age.astype(int)
df['Age'].unique()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['SSN'].unique()

- Social Security Number (SSN) is one of the features in credit scoring dataset that refers to the national identification number used by United States citizens. This number is unique for each individual and is used by the US government to identify and track individual financial and tax information.
- The values in the SSN feature will only remove the "-" symbol for ease of reading, while Null values will be left as is.

In [ ]:
df['SSN'] = df['SSN'].apply(lambda x : str(x).replace('-', '') if pd.notnull(x) else x)
df['SSN'].unique()

- For the SSN feature, it is left as an Object data type because its numerical values will lose their original meaning if it is converted to a float data type, and it cannot be converted to an integer data type because there are still NaN values present.
- The NaN values will be filled in according to their respective groups and will be converted to integers in the subsequent steps.

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
df['Annual_Income'] = df['Annual_Income'].astype(float)
df['Num_of_Loan'] = df['Num_of_Loan'].astype(int)
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].astype(float)
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].astype(float)
df['Outstanding_Debt'] = df['Outstanding_Debt'].astype(float)
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].astype(float)
df['Monthly_Balance'] = df['Monthly_Balance'].astype(float)

- The features above are clearly features that should be in numeric format.

In [ ]:
df.select_dtypes(include=['object']).info()

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

## Feature Engineering

### `Credit_History_Age`

In [ ]:
df['Credit_History_Age'].unique()

- The `Credit_History_Age` feature will be converted into a month format.

In [ ]:
def Month_Converter(x):
    if pd.notnull(x):
        list_history = x.split(' ') # ['22', 'Years', 'and', '1', 'Months']
        months = int(list_history[0])*12 + int(list_history[3]) # Taking index 0 and 3 and then adding them up based on the month format
        return months
    else:
        return x

In [ ]:
df['Credit_History_Age'] = df['Credit_History_Age'].apply(lambda x : Month_Converter(x)).astype(float)
df['Credit_History_Age'].unique()

In [ ]:
df.head(3)

### `Type_of_Loan`

In [ ]:
df['Type_of_Loan'].unique()

In [ ]:
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)

In [ ]:
df.groupby('Customer_ID')['Type_of_Loan'].apply(list)

In [ ]:
def get_Diff_Values_Colum(df_column, diff_value = []):   
    column = df_column.dropna()
    for i in column:
        if ',' not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), i.replace('and','').split(',')):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))

In [ ]:
get_Diff_Values_Colum(df['Type_of_Loan'])

## Handling Missing Values - Categorical Datatype

- It should be remembered that each set of 12 data represents a customer's credit performance in 12 months. Therefore, for some columns, the values should be the same. Hence, filling null values is based on personal customer grouping.

In [ ]:
df_object = df.select_dtypes(include=['object'])
df_object.isna().sum().sort_values(ascending=False)

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

- The function above is a function that I found on Kaggle with the source as follows: <p>
https://www.kaggle.com/code/clkmuhammed/credit-score-classification-part-1-data-cleaning#A.-Reassign-Group-NaN-Values-in-Object-Column

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
def fill_group_cats_nan(df, column, inplace=True):
    
    def fill_with_mode(df, column):
        result = df.groupby('Customer_ID')[column].apply(lambda x : x.fillna(x.mode()[0]))
        return result
    
    if inplace:  
        
        print('-' * 50)
        before = df.groupby('Customer_ID')[column].apply(list) 
        print(f'\nBefore Filling NaN {column}:\n', *before.head().values, sep='\n', end='\n')
        print('')
        display(df[column].value_counts(dropna=False))
    
        df[column] = fill_with_mode(df, column)
        
        print('=' * 50)
        after = df.groupby('Customer_ID')[column].apply(list)
        print(f'\nAfter Filling NaN {column}:\n', *after.head().values, sep='\n', end='\n')
        print('')
        display(df[column].value_counts(dropna=False))
        print('-' * 50)
    
    else:
        return fill_with_mode(df, column)

- The code that I am going to use is the code that I created inspired by the code from the previous source to avoid plagiarism.

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

### `Credit_Mix`

- `Credit_Mix` refers to the variety or diversity of credit products or accounts that a borrower has in their credit history.

In [ ]:
fill_group_cats_nan(df,'Credit_Mix')

### `Type_of_Loan`

- `Type_of_Loan` refers to the type of credit product or loan that the borrower has applied for or obtained.
- For the Null value in the `Type_of_Loan` feature, it will be replaced with 'No Data' because a customer's loan type can change every month, so using the mode will not be suitable.

In [ ]:
print('-' * 50)
print('Before Replacing NaN:')
display(df['Type_of_Loan'].value_counts(dropna=False))

df['Type_of_Loan'].replace([np.NaN], 'No Data', inplace=True)
print('=' * 50)
print('After Replacing NaN:')
display(df['Type_of_Loan'].value_counts(dropna=False))
print('-' * 50)

### `Name`

- `Name` refers to the name of the borrower or the lender who is associated with the credit account. 

In [ ]:
fill_group_cats_nan(df,'Name')

### `Payment_Behaviour`

- `Payment_Behaviour` refers to the borrower's payment history on their credit accounts.

In [ ]:
fill_group_cats_nan(df,'Payment_Behaviour')

### `Occupation`

- `Occupation` refers to the type of job or profession that the borrower has or the industry in which they work.

In [ ]:
fill_group_cats_nan(df,'Occupation')

### `SSN`

- `SSN` refers to the borrower's Social Security Number, a unique nine-digit identifier issued by the Social Security Administration (SSA) in the United States.

In [ ]:
fill_group_cats_nan(df,'SSN')

### `Payment_of_Min_Amount`

The feature `Payment_of_Min_Amount` indicates whether a customer has paid the minimum amount due on their monthly bill or not.
- "Yes" = The customer has paid the minimum amount due,
- "No" = The customer has not paid the minimum amount due, and
- "NM" or "No Data" = The information is not available for that customer.

In [ ]:
df['Payment_of_Min_Amount'].value_counts(dropna=False)

- Since this feature does not have any Null value, there is no need to apply any treatment.

### Final Check - 1

In [ ]:
df.select_dtypes(include=['object']).isna().sum().sort_values(ascending=False)

- From the output above, we know that there are no more missing values that need to be cleaned in categorical features. Note that for the `Credit_Score`feature, it is part of the test data so there is no need to clean its null values.

## Handling Error and Missing Values - Numerical Datatype

In [ ]:
df.select_dtypes(include=['int32','int64','float64']).isna().sum().sort_values(ascending=False)

In [ ]:
df.describe()

- From the above data, it appears that there are columns (`Age`, `Num_of_Delayed_Payment`, etc) with erroneous values, such as the minimum `Age` of customers being -500 and the maximum being 8698. Therefore, it is necessary to correct these values by aligning them with the values of their respective groups based on the `Customer_ID`.

df.to_excel('DC_ManualCheck.xlsx', index=False)

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

In [ ]:
# Reassign Wrong Values and Show Function
def Reassign_Group_Numeric_Wrong_Values(df, groupby, column, inplace=True):      
    import scipy.stats as stats 

    # Identify Wrong values Range
    def get_group_min_max(df, groupby, column):            
        cur = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = cur.apply(lambda x: stats.mode(x)).apply([min, max])
        return x[0][0], y[0][0]
    
    # Assigning Wrong values
    def make_group_NaN_and_fill_mode(df, groupby, column, inplace=True):
        df_dropped = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = df_dropped.apply(lambda x: stats.mode(x)).apply([min, max])
        mini, maxi = x[0][0], y[0][0]

        # assign Wrong Values to NaN
        col = df[column].apply(lambda x: np.NaN if ((x<mini)|(x>maxi)) else x)

        # fill with local mode
        mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
        result = col.fillna(mode_by_group)

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
        
    
    # Run      
    if inplace:
        print('=' * 50)
        print("\nExisting Min, Max Values:", df[column].apply([min, max]).rename({'min': 'min:', 'max': 'max:'}), sep='\n', end='\n')
        
        mini, maxi = get_group_min_max(df, groupby, column)
        
        print('')
        print('-' * 50)
        print(f"\nGroupby by {groupby}'s Actual min, max Values:", f'min:\t{mini},\nmax:\t{ maxi}', sep='\n', end='\n')
        print('')
        print('=' * 50)
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_group_NaN_and_fill_mode(df, groupby, column, inplace)
        
        print('')
        print('-' * 50)
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
        print('=' * 50)
    else:   
        # Show
        return make_group_NaN_and_fill_mode(df, groupby, column, inplace)

- The function above is a function that I found on Kaggle with the source as follows: <p>
https://www.kaggle.com/code/clkmuhammed/credit-score-classification-part-1-data-cleaning#B.-Detect-Group-Min-Max-in-Numeric-Column-Reassign-NaN-Values

- Since I am not yet skilled enough to create complex code, I will use the code above. Although I do not fully understand some parts of this code, I still use it because the results are already in line with what I want. Of course, I will continue to improve my skills in the future to become a Data Scientist with advanced coding skills. Thank you.

<div style="text-align: left; width: 50%; display: inline-block;">
    <hr style="border: 1px solid white;">
</div>

### `Monthly_Inhand_Salary`

- `Monthly_Inhand_Salary` refers to the amount of money that the borrower earns each month after tax and other deductions have been taken out of their gross salary.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Inhand_Salary')

### `Credit_History_Age`

- `Credit_History_Age` refers to the length of time that the borrower has had credit accounts or loans in their credit history.

In [ ]:
df['Credit_History_Age'].value_counts(dropna=False)

In [ ]:
df.groupby('Customer_ID')['Credit_History_Age'].value_counts(dropna=False)

- As it is known that Credit_History_Age changes according to the month for each customer, filling Null values with Mode is not suitable.
- The use of interpolation is suitable for filling in missing values in each customer group. Moreover, linear interpolation is very suitable for the context of credit history age according to the month. However, if interpolation still leaves missing values because the missing data is at the beginning or end of the data, the back fill and forward fill methods can be a solution.
- The forward fill function will be executed first considering the context of the values in the column will continue to increase as the months change.

In [ ]:
df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].apply(lambda x : x.interpolate().ffill().bfill())

### `Num_of_Delayed_Payment`

- `Num_of_Delayed_Payment` refers to the number of times that the borrower has made late payments on their credit accounts or loans. 

In [ ]:
df['Num_of_Delayed_Payment'].value_counts()

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Delayed_Payment')

### `Amount_invested_monthly `

- `Amount_invested_monthly` refers to the amount of money that a borrower invests in savings or other investment accounts each month.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Amount_invested_monthly')

- The above output shows that the `maximum` value of the feature `Amount_invested_monthly` is much larger compared to the other groups. Outliers like this can cause the data distribution to be skewed.

### `Changed_Credit_Limit`

- `Changed_Credit_Limit` refers to any changes to a borrower's credit limit on their credit accounts or loans.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Changed_Credit_Limit')

### `Num_Credit_Inquiries`

- `Num_Credit_Inquiries` refers to the number of times that a borrower's credit report has been accessed by lenders or other entities to evaluate the borrower's creditworthiness.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Inquiries')

### `Monthly_Balance`

- `Monthly_Balance` refers to the balance or amount owed by the borrower on their credit accounts or loans at the end of each month.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')

- The above output shows that the `minimum` value of the feature `Monthly_Balance` is much larger compared to the other groups. Outliers like this can cause the data distribution to be skewed.

### Final Check - 2

In [ ]:
df.select_dtypes(include=['int32','int64','float64']).isna().sum().sort_values(ascending=False)

- Although there are no more Null values in the data, there are still several columns that need to be corrected because they have some unusual values that do not match the context as previously discussed.

## Handling Error Values - Numerical Datatype

In [ ]:
df.describe()

In [ ]:
Error_Value_Columns = ['Total_EMI_per_month', 'Num_of_Loan', 'Interest_Rate', 'Num_Credit_Card', 
                       'Num_Bank_Accounts', 'Annual_Income', 'Age', 'Delay_from_due_date']

for col in Error_Value_Columns:
    print(col)

- The data cleaning process will use the same function as in the previous process.

### `Total_EMI_per_month`

- `Total_EMI_per_month` is the total monthly installment amount (including interest) that a person has to pay for all their loans or credits in a month. EMI stands for Equated Monthly Installment, which is typically used to pay off loans such as car loans or home loans.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Total_EMI_per_month')

### `Num_of_Loan`

- `Num_of_Loan` represents the number of loans taken by each individual in the dataset.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Loan')

### `Interest_Rate`

- `Interest_Rate` represents the interest rate charged on a loan. It is usually expressed as a percentage of the total loan amount and is the additional amount that a borrower has to pay on top of the principal amount borrowed.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Interest_Rate')

### `Num_Credit_Card`

- `Num_Credit_Card` represents the number of credit cards that the individual has.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Card')

### `Num_Bank_Accounts`

- `Num_Bank_Accounts` represents the number of bank accounts that the borrower has.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Bank_Accounts')

### `Annual_Income`

- `Annual_Income` is the annual income of individuals or entities. It is typically expressed in a numerical value and represents the total amount of money earned by a person or business in a given year.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Annual_Income')

### `Age`

- `Age` refers to the chronological age of the borrower, measured in years. 

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Age')

### `Delay_from_due_date`

- `Delay_from_due_date` is the number of days of delay from the due date of a payment.
- At first, this column had a maximum value that was still considered reasonable, but because it had a value of -5 where there was already -1, it felt quite strange for the Delay column which usually has a positive value conversion to explain the number of days of delay from the due date.

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Delay_from_due_date')

### Final Check - 3

In [ ]:
df.info()

- The cleaning process is almost complete. However, there are still some things that need to be changed, namely the `maximum` value of `Amount_invested_monthly` and the `minimum` value of `Monthly_Balance` which are suspected to be `global outliers` or may be due to human error.

## Handling Outliers

In [ ]:
numerical_features = df.select_dtypes(include=np.number).columns.tolist()
print(numerical_features)
print('')
print('Total Numerical Features:',len(numerical_features))

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(20,10))

for i, col in enumerate(numerical_features):
    sns.boxplot(x=df[col], color='g', ax=axs[i//5, i%5])
    
plt.tight_layout()
plt.show()

### `Amount_invested_monthly` - 2

In [ ]:
df[df['Amount_invested_monthly'] > 8000]

In [ ]:
df.loc[df['Amount_invested_monthly'] > 8000, 'Customer_ID'].nunique()

In [ ]:
df.loc[df['Amount_invested_monthly'] > 8000, 'Customer_ID'].unique().tolist()

In [ ]:
print('Customer_ID = 33982')
print('-'*20)
display(df[df['Customer_ID'] == 33982]['Amount_invested_monthly'])

- From the data, it is apparent that there is a value that is significantly higher than the others. In real-life contexts, this can happen, especially when no one knows a person's financial situation specifically. However, to avoid inaccurate models for the entire dataset, outliers like this need to be handled to improve the data distribution.
- The method I chose is to handle outliers with a robust method or to replace the value with the median value of each data group based on each `Customer_ID`.
- I am aware that filtering outliers using the IQR method cannot be applied to all data per `Customer_ID` group, considering that the example above only has 4 out of 12 data per customer with invalid values, which means those values fall into Q3.
- Therefore, I directly filtered each number `above 8000` (obtained from the previous boxplot visualization) to be replaced with the median per `Customer_ID` group.

In [ ]:
grouped = df.groupby('Customer_ID')

# Loop through each Customer_ID group
for name, group in grouped:
    # Determining median (excluding values greater than 8000)
    median = group.loc[group['Amount_invested_monthly'] <= 8000, 'Amount_invested_monthly'].median()
    
    # Replacing values greater than 8000 with median value
    group.loc[group['Amount_invested_monthly'] > 8000, 'Amount_invested_monthly'] = median
    
    # Saving changes to the DataFrame
    df.loc[group.index] = group

In [ ]:
print('Customer_ID = 33982')
print('-'*20)
display(df[df['Customer_ID'] == 33982]['Amount_invested_monthly'])

### `Monthly_Balance` - 2

In [ ]:
df[df['Monthly_Balance'] < 0]

In [ ]:
df.loc[df['Monthly_Balance'] < 0, 'Customer_ID'].unique()

In [ ]:
df[df['Customer_ID'] == 39045]['Monthly_Balance']

- The data at `index 5545` above is clearly invalid. The feature `Monthly_Balance` cannot have a negative value. This is also not a labeling error, considering the exponential value behind it is significantly larger than the other values.
- The same robust method as before will be used to replace the outlier data assumed to be invalid data. However, this time outliers will be filtered using the IQR method to have a clearer basis.

In [ ]:
grouped = df.groupby('Customer_ID')

# Loop through each Customer_ID group
for name, group in grouped:
    # Determining IQR
    Q1 = group['Monthly_Balance'].quantile(0.25)
    Q3 = group['Monthly_Balance'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Replacing outlier values with median values
    group.loc[group['Monthly_Balance'] > Q3 + 1.5*IQR, 'Monthly_Balance'] = group['Monthly_Balance'].median()
    group.loc[group['Monthly_Balance'] < Q1 - 1.5*IQR, 'Monthly_Balance'] = group['Monthly_Balance'].median()
    
    # Saving changes to the DataFrame
    df.loc[group.index] = group

In [ ]:
df[df['Customer_ID'] == 39045]['Monthly_Balance']

### Final Check - 4

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(20,10))

for i, col in enumerate(numerical_features):
    sns.boxplot(x=df[col], color='g', ax=axs[i//5, i%5])
    
plt.tight_layout()
plt.show()

In [ ]:
df.shape

# EDA (EXPLORATORY DATA ANALYSIS)

In [ ]:
train_data = df[df['Credit_Score'].notnull()]
test_data = df[df['Credit_Score'].isnull()]

print('Total train set:', train_data.shape)
print('Total test set:', test_data.shape)

In [ ]:
train_data.sample(3)

In [ ]:
test_data.sample(3)

In [ ]:
train_data.info()


Untuk melakukan EDA dengan menggunakan line chart, scatter plot, dan barplot, berikut beberapa rekomendasi antar kolom yang bisa dipertimbangkan:

Line Chart:
- Monthly_Inhand_Salary berdasarkan bulan (Month)
- Amount_invested_monthly berdasarkan bulan (Month)
- Credit_Score berdasarkan bulan (Month)

Scatter Plot:
- Monthly_Inhand_Salary vs Annual_Income
- Monthly_Balance vs Outstanding_Debt
- Amount_invested_monthly vs Total_EMI_per_month

Barplot:
- Num_Credit_Card berdasarkan Type_of_Loan
- Num_of_Loan berdasarkan Type_of_Loan
- Payment_of_Min_Amount berdasarkan Payment_Behaviour

Namun demikian, rekomendasi antar kolom tersebut bisa disesuaikan dengan tujuan EDA dan pertanyaan yang ingin dijawab dari data tersebut.

Beberapa contoh pertanyaan bisnis yang sering ingin dijawab dengan proyek machine learning credit scoring adalah:

- Bagaimana cara meningkatkan akurasi dalam menilai risiko kredit?
- Apa faktor-faktor yang paling mempengaruhi kemungkinan seseorang gagal membayar kredit?
- Bagaimana cara meminimalkan risiko default pada pemberian kredit?
- Apakah terdapat bias dalam model credit scoring dan bagaimana cara mengurangi atau menghilangkan bias tersebut?
- Bagaimana memprediksi apakah calon peminjam akan membayar kembali pinjaman tepat waktu atau tidak?
- Bagaimana cara memperkirakan besarnya risiko default untuk masing-masing pelanggan?
- Apakah ada pola atau hubungan antara karakteristik pelanggan dengan kemampuan mereka untuk membayar pinjaman?
- Bagaimana cara memilih metode pembelajaran mesin yang paling cocok untuk memprediksi kemampuan seseorang dalam membayar kredit?
- Bagaimana cara menentukan ukuran sampel yang optimal untuk model credit scoring?
- Bagaimana cara mengukur kinerja model credit scoring dan bagaimana cara meningkatkan performa model tersebut?

-----

Berikut adalah beberapa rekomendasi antara kolom untuk ditampilkan dalam proses EDA insight sebelum melakukan preprocessing ML untuk menjawab pertanyaan "Bagaimana cara meminimalkan risiko default pada pemberian kredit?":

Line chart

- "Credit_History_Age" dan "Credit_Score"
Alasan: Dengan menampilkan hubungan antara usia kredit dan skor kredit, kita dapat melihat bagaimana umur kredit memengaruhi risiko default pada pemberian kredit.

- "Monthly_Inhand_Salary" dan "Total_EMI_per_month"
Alasan: Dengan menampilkan hubungan antara gaji bersih bulanan dan total EMI per bulan, kita dapat melihat bagaimana kemampuan pembayaran peminjam memengaruhi risiko default pada pemberian kredit.

Scatter plot 
- "Credit_Utilization_Ratio" dan "Credit_Score"
Alasan: Dengan menampilkan hubungan antara rasio penggunaan kredit dan skor kredit, kita dapat melihat bagaimana penggunaan kredit memengaruhi risiko default pada pemberian kredit.

- "Outstanding_Debt" dan "Num_of_Delayed_Payment"
Alasan: Dengan menampilkan hubungan antara hutang yang masih harus dibayar dan jumlah pembayaran yang tertunda, kita dapat melihat bagaimana situasi keuangan peminjam memengaruhi risiko default pada pemberian kredit.

Barplot
- "Type_of_Loan" dan "Interest_Rate"
Alasan: Dengan menampilkan perbedaan suku bunga antara jenis pinjaman yang berbeda, kita dapat melihat bagaimana risiko default pada jenis pinjaman yang berbeda dapat mempengaruhi suku bunga yang ditetapkan.

- "Payment_Behaviour" dan "Num_of_Delayed_Payment"
Alasan: Dengan menampilkan jumlah pembayaran yang tertunda untuk setiap perilaku pembayaran, kita dapat melihat bagaimana perilaku pembayaran memengaruhi risiko default pada pemberian kredit.

## WOE and IV

# MODELLING

- In a credit scoring dataset, the Credit_Score feature is the target of classification. Therefore, the dataset will be divided into X and y where X is the predictor features and y is the target.

## Split Dataset

- The data that will be used will be divided into train and test sets. However, the division will still be sourced from the `train_data` in which the target feature `Credit_Score` <B>is not null</B>. 
- This is because if `test_data` is used for X_test and y_test, it <B>will not produce any evaluation</B>. The `test_data` data will be used when we really want to predict the output assuming that the data is new data from each customer whose `Credit_Score` is to be predicted.

In [ ]:
from sklearn.model_selection import train_test_split

X = train_data.drop('Credit_Score', axis=1)
y = train_data['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

In [96]:
print('X_train :', X_train.shape)
print('y_train :', y_train.shape)
print('-'*20)
print('X_test :', X_test.shape)
print('y_test :', y_test.shape)
print('='*20)
print('Total y_train')
display(y_train.value_counts())
print('-'*20)
print('Total y_test')
display(y_test.value_counts())

X_train : (70000, 27)
y_train : (70000,)
--------------------
X_test : (30000, 27)
y_test : (30000,)
Total y_train


Standard    37267
Poor        20265
Good        12468
Name: Credit_Score, dtype: int64

--------------------
Total y_test


Standard    15907
Poor         8733
Good         5360
Name: Credit_Score, dtype: int64

## PreProcessing

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int32', 'int64', 'float64']).columns

---

In [95]:
X_try = test_data.drop('Credit_Score', axis=1)
y_try = test_data['Credit_Score']

In [96]:
print('X_test :', X_test.shape)
print('y_test :', y_test.shape)
print('-'*20)
display(y_test.value_counts())

X_test : (50000, 27)
y_test : (50000,)
--------------------


Series([], Name: Credit_Score, dtype: int64)